## 1. Import

In [ ]:
import pandas as pd
import numpy as np

#StopWords Removal
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

import re

from sklearn.model_selection import train_test_split
import nltk

import textblob
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer


import torch
from tqdm.notebook import tqdm

from transformers import BertTokenizer
from torch.utils.data import TensorDataset

from transformers import BertForSequenceClassification

Import Data Set

In [ ]:
McDonalds_Df_Raw = pd.read_csv("McDonald_s_Reviews.csv", encoding="latin-1")
McDonalds_Df_Raw

## 2. Organize Data

Remove unnecessary information

In [ ]:
McDonalds_Df_Raw = McDonalds_Df_Raw.set_index("reviewer_id")
McDonalds_Df_Raw = McDonalds_Df_Raw.drop(columns = ["store_name", "category"])
McDonalds_Df_Raw

## 3. Keywords Frequency and Ratings

Select columns needed

In [ ]:
df_keyword_ratings = McDonalds_Df_Raw.get(["review", "rating"])
df_keyword_ratings

Process dataframe

In [ ]:
df_keyword_ratings = df_keyword_ratings.assign(rating = McDonalds_Df_Raw.get("rating").apply(lambda x: int(x[0])))
df_keyword_ratings = df_keyword_ratings.sort_values(by = ["rating"])
df_keyword_ratings

4. Removing Stopwords

In [ ]:

import nltk

stop_words = stopwords.words('english')
def remove_stop_words(review):
    '''
    Takes in a string comment or review returns a string that has the stopwords removed. For better comprehension of keywords.
    '''
    phrase = review.split()
    stripped_phrase = []
    for word in phrase:
        if word not in stop_words:
            stripped_phrase.append(word)

    return " ".join(stripped_phrase)


In [ ]:
def remove_undecoded(review):
    '''
    removes the undecoded texts such as "�" in the original csv file. 
    '''
    return re.sub('[^0-9a-zA-Z\s]+', "",review)
    


In [ ]:
#df_nostop_keywords = df_keyword_ratings.assign(no_stop_words_review = df_keyword_ratings.get("review").apply(remove_stop_words))
df_nostop_keywords = df_keyword_ratings.assign(no_stop_words_review = df_keyword_ratings.get("review").apply(remove_undecoded))
df_nostop_keywords = df_nostop_keywords.drop(columns = "review")

# Exporting the processed file into CSV
df_nostop_keywords.to_csv("review_without_stop_words", encoding = "latin-1")

In [ ]:
# df to array put array
# import the training test split. 80 20, 60 40 70 30 <- use package
#
review = np.array(df_nostop_keywords.get("no_stop_words_review"))

In [ ]:


def sentimentation(sentences):
    analyzer = SentimentIntensityAnalyzer()
    #compound_score = sentiment['compound']
    # Break down the text into sentences
    sentences = nltk.sent_tokenize(sentences)

    # Analyze sentiment for each sentence
    for sentence in sentences:
        sentiment = analyzer.polarity_scores(sentence)

        return sentiment['compound'] 

In [57]:
df = df_nostop_keywords.sample(n=100)

#df = df_sentiment_nostop_keywords.assign(calculated_sent = df_sentiment_nostop_keywords.get("no_stop_words_review").apply(sentimentation))

df['data_type'] = ['not_set']*df.shape[0]
df = df.set_index("no_stop_words_review")
df["rating"]= df["rating"].astype("string")
X_train, X_test, y_train, y_test = train_test_split(df.index.values, df.get("rating").values, test_size=0.50, stratify=y)#change back to 0.20

df.loc[X_train, 'data_type'] = 'train'
df.loc[X_test, 'data_type'] = 'val'

df = df.reset_index()

df.groupby(['rating', 'data_type']).count()


no_stop_words_review
rating data_type                      
1      train                        15
       val                          16
2      train                         6
       val                          11
3      train                         5
       val                           8
4      train                         6
       val                           8
5      train                        12
       val                          13

In [58]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', 
                                          do_lower_case=True)
                                          
encoded_data_train = tokenizer.batch_encode_plus(
    df[df.data_type=='train'].index.values, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    pad_to_max_length=True, 
    max_length=256, 
    return_tensors='pt'
)

encoded_data_val = tokenizer.batch_encode_plus(
    df[df.data_type=='val'].index.values, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    pad_to_max_length=True, 
    max_length=256, 
    return_tensors='pt'
)


input_ids_train = encoded_data_train['input_ids']
attention_masks_train = encoded_data_train['attention_mask']
labels_train = torch.tensor(df[df.data_type=='train'].get("rating").values)

input_ids_val = encoded_data_val['input_ids']
attention_masks_val = encoded_data_val['attention_mask']
labels_val = torch.tensor(df[df.data_type=='val'].get("rating").values)

dataset_train = TensorDataset(input_ids_train, attention_masks_train, labels_train)
dataset_val = TensorDataset(input_ids_val, attention_masks_val, labels_val)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
c:\Users\theso\AppData\Local\Programs\Python\Python39\lib\site-packages\transformers\tokenization_utils_base.py:2418: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


ValueError: Input is not valid. Should be a string, a list/tuple of strings or a list/tuple of integers.

In [ ]:
# df_rating_stratfied = df_nostop_keywords.set_index()[df_nostop_keywords.get("rating") == 1]
# df_rating_stratifiedd

In [ ]:
# np.random.seed(1)
# df_sampled_1 = df_rating_1.sample(n=10)
# df_sampled_1

In [ ]:
# df_sampled_1.assign(sentiment = df_sampled_1.get("no_stop_words_review").apply(sentimentation))

In [ ]:
# df_rating_2 = df_nostop_keywords[df_nostop_keywords.get("rating") == 2]
# df_rating_2

In [ ]:
# np.random.seed(1)
# df_sampled_2 = df_rating_2.sample(n=10)
# df_sampled_2

In [ ]:
# df_sampled_2.assign(sentiment = df_sampled_2.get("no_stop_words_review").apply(sentimentation))